# Praca projektowa nr 4
# Maciej Chylak
# 305699 IiAD

<br/>
<br/>

## Przygotowanie ramek danych do użycia

In [115]:
import pandas as pd
import numpy as np
Tags = pd.read_csv("travel_stackexchange_com/Tags.csv")
Badges = pd.read_csv("travel_stackexchange_com/Badges.csv")
Comments = pd.read_csv("travel_stackexchange_com/Comments.csv")
Posts = pd.read_csv("travel_stackexchange_com/Posts.csv")
PostLinks = pd.read_csv("travel_stackexchange_com/PostLinks.csv")
Users = pd.read_csv("travel_stackexchange_com/Users.csv")
Votes = pd.read_csv("travel_stackexchange_com/Votes.csv")

In [116]:
import os, os.path
import sqlite3
import tempfile
# sciezka dostępu do bazy danych:
baza = os.path.join(tempfile.mkdtemp(), 'przyklad.db')
if os.path.isfile(baza): # jesli baza już istneje...
    os.remove(baza) # ...usuniemy ja
conn = sqlite3.connect(baza) # połączenie do bazy danych
Badges.to_sql("Badges", conn) # importujemy ramkę danych do bazy danych
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

***

## Zadanie nr 1

#### SQL 

In [121]:
df_sql_1 = pd.read_sql_query(""" SELECT Title, Score, ViewCount, 
FavoriteCount FROM Posts WHERE PostTypeId=1 AND FavoriteCount >= 25 AND ViewCount>=10000 """, conn)

#### PANDAS

In [136]:
df_pd_1 = Posts[(Posts.PostTypeId == 1) & (Posts.FavoriteCount >= 25) & (Posts.ViewCount >= 10000)][['Title', 'Score', 'ViewCount', 'FavoriteCount']]
df_pd_1 = df_pd_1.reset_index(drop = True)

***

## Zadanie nr 2

#### SQL 

In [123]:
df_sql_2 = pd.read_sql_query(""" SELECT Posts.Title, RelatedTab.NumLinks FROM
(SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
FROM PostLinks GROUP BY RelatedPostId) AS RelatedTab
JOIN Posts ON RelatedTab.PostId=Posts.Id
WHERE Posts.PostTypeId=1
ORDER BY NumLinks DESC
LIMIT 10 """, conn)

#### PANDAS

In [127]:
RelatedTab = PostLinks.groupby('RelatedPostId').size().to_frame("NumLinks").reset_index()
RelatedTab = RelatedTab.rename(columns={'RelatedPostId':'PostId'})
df_pd_2 = RelatedTab.merge(
    Posts[(Posts.PostTypeId == 1)], left_on = "PostId", 
    right_on = "Id").sort_values(['NumLinks'], ascending=[False]).head(10)
df_pd_2 = df_pd_2[["Title", "NumLinks"]].reset_index(drop = True)

***

## Zadanie nr 3

#### SQL 

In [128]:
df_sql_3 = pd.read_sql_query(""" SELECT DISTINCT
Users.Id,
Users.DisplayName,
Users.Reputation,
Users.Age,
Users.Location
FROM (
SELECT
Name, UserID
FROM Badges
WHERE Name IN (
SELECT
Name
FROM Badges
WHERE Class=1
GROUP BY Name
HAVING COUNT(*) BETWEEN 2 AND 10
)
AND Class=1
) AS ValuableBadges
JOIN Users ON ValuableBadges.UserId=Users.Id """, conn)

#### PANDAS

In [134]:
x = Badges[(Badges.Class == 1)].groupby("Name").size().to_frame("n").reset_index()
x = x[(x.n >=2) & (x.n <= 10)]["Name"]
ValuableBadges = Badges[(Badges.Name.isin(x)) & (Badges.Class == 1)][['Name', 'UserId']]
df_pd_3 = ValuableBadges.merge(Users, left_on = "UserId", right_on = "Id")[["Id", "DisplayName", "Reputation", "Age", "Location"]]
df_pd_3 = df_pd_3.drop_duplicates().reset_index(drop = True)

***

## Sprawdzenie poprawności 

In [137]:
df_pd_1.equals(df_sql_1)

True

In [138]:
df_pd_2.equals(df_sql_2)

True

In [139]:
df_pd_3.equals(df_sql_3)

True